## 3.3 通过自注意力机制关注输入的不同部分

![from comp5046](imgs/PixPin_2025-07-06_04-33-57.png)

**Self-Attention**: A weighted average of vectors where the weights and input vectors are all from the input

![Alt text](imgs/PixPin_2025-07-06_04-36-28.png)

图 3.7 显示了一个输入序列，记作 x，由 T 个元素组成，表示为 x(1) 到 x(T)。该序列通常代表文本，例如一个句子，并且该文本已被转换为 token 嵌入。

举例来说，假设输入文本为 “Your journey starts with one step”。在这个例子中，序列中的每个元素（如 x(1)）对应一个 d 维的嵌入向量，用于表示特定的 token，例如 “Your”。在图 3.7 中，这些输入向量显示为 3 维的嵌入向量。

在自注意力机制中，我们的**目标**是为输入序列中的每个元素 x(i) 计算其对应的上下文向量 z(i) 。为了说明这个概念，我们聚焦于第二个输入元素 x(2) 的嵌入向量（对应于词 "journey"）以及相应的上下文向量 z(2)，如图 3.7 底部所示。这个增强的上下文向量 z(2) 也是一个嵌入向量，包含了关于 x(2) 以及序列中所有其他输入元素 x(1) 到 x(T) 的语义信息。

In [1]:
import torch
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)


![Alt text](imgs/PixPin_2025-07-06_04-40-41.png)

这里qi是Gelato，kj可以是is或者delicious。但还有问题，首先weighted是线性的，并且这里并没有词序的关系,因此我们引入了之前提到的位置编码。

![Alt text](imgs/PixPin_2025-07-06_04-41-22.png)

![Alt text](imgs/PixPin_2025-07-06_04-42-44.png)

实现自注意力机制的第一步是计算中间值 ω，即注意力得分，如图 3.8 所示。（请注意，图 3.8 中展示的输入张量值是截断版的，例如，由于空间限制，0.87 被截断为 0.8。在此截断版中，单词 "journey" 和 "starts" 的嵌入向量可能会由于随机因素而看起来相似）。

![Alt text](imgs/PixPin_2025-07-06_04-43-28.png)


根据input，我们可以设计一个小demo：
首先，我们要制定Q，K，V的权重矩阵。

In [2]:
W_Q = torch.tensor([[0.1, 0.3, 0.5],  # 3x3
                    [0.2, 0.4, 0.6],
                    [0.3, 0.5, 0.7]])

W_K = torch.tensor([[0.2, 0.1, 0.4],
                    [0.3, 0.2, 0.5],
                    [0.4, 0.3, 0.6]])

W_V = torch.tensor([[0.5, 0.4, 0.3],
                    [0.6, 0.5, 0.2],
                    [0.7, 0.6, 0.1]])


然后，计算绝对位置编码。

In [5]:
import torch.nn.functional as F

def positional_encoding(pos, d_model):
    pe = torch.zeros(d_model)
    for i in range(0, d_model, 2):
        angle = torch.tensor(pos / (10000 ** ((2 * i)/d_model)))
        pe[i] = torch.sin(angle)

        if i + 1 < d_model:
            angle = torch.tensor(pos / (10000 ** ((2 * i)/d_model)))
            pe[i+1] = torch.cos(angle)
    return pe
    
pos_encodings = torch.stack([positional_encoding(pos, inputs.shape[1]) for pos in range(1, 7)])

inputs_with_pe = inputs + pos_encodings

In [7]:
# 计算 Q, K, V
Q = inputs_with_pe @ W_Q  # (6,3)
K = inputs_with_pe @ W_K  # (6,3)
V = inputs_with_pe @ W_V  # (6,3)

# 计算 attention scores (Q @ K.T / sqrt(d_k))
d_k = Q.shape[1]
scores = Q @ K.T / torch.sqrt(torch.tensor(d_k, dtype=torch.float32))

# Softmax 归一化
attention_weights = F.softmax(scores, dim=-1)

# 加权求和
output = attention_weights @ V

import pandas as pd

# 拼接输入、注意力权重、输出结果
df_inputs = pd.DataFrame(inputs_with_pe.numpy(), columns=['x1', 'x2', 'x3'])
df_attention = pd.DataFrame(attention_weights.numpy(), columns=[f"T{i+1}" for i in range(6)])
df_output = pd.DataFrame(output.numpy(), columns=['o1', 'o2', 'o3'])

# 合并所有 DataFrame
result_df = pd.concat([df_inputs, df_attention, df_output], axis=1)

# 打印结果
print("\n===== Self-Attention 结果 (含位置编码) =====\n")
print(result_df.to_string(index=False))



===== Self-Attention 结果 (含位置编码) =====

       x1        x2       x3       T1       T2       T3       T4       T5       T6       o1       o2       o3
 1.271471  0.690302 0.890005 0.315381 0.239435 0.105702 0.044346 0.066093 0.229044 1.273928 1.060221 0.435727
 1.459297  0.453853 0.660009 0.296198 0.233471 0.115046 0.054239 0.076627 0.224420 1.236005 1.028942 0.420495
 0.711120 -0.139992 0.640014 0.231497 0.205499 0.144342 0.099181 0.117838 0.201644 1.086208 0.905413 0.360154
-0.536803 -0.073644 0.330019 0.162206 0.163468 0.167715 0.172386 0.170190 0.164035 0.885203 0.739702 0.278801
-0.188924  0.533662 0.100023 0.193902 0.184244 0.158497 0.135087 0.145366 0.182904 0.982739 0.820102 0.318358
-0.229415  1.760170 0.550028 0.288950 0.230573 0.118408 0.058350 0.080746 0.222973 1.221021 1.016597 0.414371


```python
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)
```

```x1, x2, x3```是原始词向量 + 绝对位置编码，即 inputs_with_pe

```xml
原始向量: [0.43, 0.15, 0.89]
位置编码: [0.84, 0.54, 0.00]
相加结果: [1.271471, 0.690302, 0.89005]
```

```T1~T6```: 注意力权重矩阵的某一行,表示当前 token 对句子中 每个 token 的关注度。

例子（第 1 个 token 的 attention 分布）：

```md
T1=0.31581  (自己: 31.6%)
T2=0.239435 (关注 token2: 23.9%)
...
T6=0.229044 (关注 token6: 22.9%)
```

```o1~o3```: Self-Attention 的最终输出向量, 即每个 token 的上下文感知表示。



回到教材：图 3.8 展示了如何计算查询 token 与每个输入 token 之间的中间注意力得分。我们通过计算查询 x(2) 与每个其他输入 token 的点积来确定这些得分（这里只是简单的将第二个token作为query vector进行了q与k的dot product）：

In [8]:
query = inputs[1]                                               #A
attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query)
print(attn_scores_2)


#A 第二个输入 token 用作查询向量

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [10]:
# 使用 softmax 函数来进行归一化。
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)
print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


> [!NOTE]
> 1. Softmax 的好处
> 
> - ```归一化输出为概率```：Softmax 将输出转换为 0 到 1 之间的概率，且所有类别的概率之和为 1，方便解释结果。例如，在分类任务中，输出可以直接表示模型对各类别的信心。
> 
> - ```平滑和放大效果```：Softmax 不仅能归一化，还具有平滑和放大效果。较大的输入值会被放大，较小的输入值会被抑制，从而增强模型对最优类别的区分。
>
> - ```支持多分类问题```：与 sigmoid 不同，Softmax 适用于多类别分类问题。它可以输出每个类别的概率，使得模型可以处理多分类任务。
>
> 2. 神经网络为什么喜欢使用 Softmax
>
> ```在神经网络中```，特别是分类模型（如图像分类、文本分类）中，Softmax 层通常用作最后一层输出。原因包括：
> 
> - ```便于优化```：在分类任务中，Softmax 输出的概率分布可与真实的标签概率进行比较，从而计算交叉熵损失。交叉熵损失的梯度较为稳定，便于模型的优化。
>
> - ```概率解释```：Softmax 输出可以解释为“模型对每个类别的信心”，使得输出直观可理解。
>
> - ```与交叉熵的结合```：Softmax 与交叉熵损失函数结合效果特别好，可以直接将模型预测的概率分布与真实标签比较，从而更快收敛，效果更好。